In [12]:
!pip install langchain langchain-openai langchain-community faiss-cpu unstructured

In [13]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [14]:
import os
import random
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# ★修正点: CSVLoaderを追加
from langchain_community.document_loaders import DirectoryLoader, TextLoader, CSVLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from google.colab import drive, userdata

# ==============================================================================
# 1. Characterクラスの定義（RAG学習ロジックを含む）
# ==============================================================================
class Character:
    """キャラクターのRAGチェーンと情報をカプセル化するクラス"""
    def __init__(self, name: str, private_docs_path: str, shared_logs_path: str, system_prompt_template: str):
        self.name = name
        print(f"キャラクター「{self.name}」を構築中...")

        # --- ドキュメント読み込み処理の改良 ---
        # 1. 個別の知識ファイル（プライベートな情報）を読み込む
        private_loader = DirectoryLoader(private_docs_path, glob="**/*.txt", loader_cls=TextLoader)
        private_docs = private_loader.load()
        print(f"  > 個別知識を {len(private_docs)} 件読み込みました。")

        # ★修正点1: 共有の会話ログから、自分自身のCSVファイルのみを読み込むように変更
        shared_docs = []
        character_log_file = os.path.join(shared_logs_path, f"{self.name}.csv")

        if os.path.exists(character_log_file):
            try:
                # 自身のキャラクター名のCSVファイルを読み込む
                log_loader = CSVLoader(file_path=character_log_file, encoding='utf-8')
                shared_docs = log_loader.load()
                print(f"  > 共有会話ログ '{os.path.basename(character_log_file)}' を読み込み、{len(shared_docs)} 件のドキュメントを取得しました。")
            except Exception as e:
                print(f"  > 共有会話ログ '{os.path.basename(character_log_file)}' の読み込み中にエラーが発生しました: {e}")
        else:
            print(f"  > 共有会話ログ '{os.path.basename(character_log_file)}' が見つかりませんでした。")

        # 3. 全てのドキュメントを結合
        all_docs = private_docs + shared_docs
        print(f"  > 合計 {len(all_docs)} 件のドキュメントで知識ベースを構築します。")
        # --- ここまでが改良箇所 ---

        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
        embeddings = OpenAIEmbeddings()

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        split_docs = text_splitter.split_documents(all_docs)
        vectorstore = FAISS.from_documents(split_docs, embeddings)
        retriever = vectorstore.as_retriever()

        # RAGチェーンの構築（会話履歴を考慮する設計）
        history_aware_prompt = ChatPromptTemplate.from_messages([
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            ("user", "上記の発言を踏まえ、関連情報を検索するためのキーワードを生成してください。"),
        ])
        history_aware_retriever = create_history_aware_retriever(llm, retriever, history_aware_prompt)

        answer_prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt_template),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
        ])
        document_chain = create_stuff_documents_chain(llm, answer_prompt)
        self.rag_chain = create_retrieval_chain(history_aware_retriever, document_chain)
        print(f"キャラクター「{self.name}」の構築完了。")

    def speak(self, input_text: str, chat_history: list):
        """与えられた入力と会話履歴に基づいて発言を生成する"""
        result = self.rag_chain.invoke({"input": input_text, "chat_history": chat_history})
        return result["answer"]

# ==============================================================================
# 2. DialogueManagerクラスの定義（対話フローを改良）
# ==============================================================================
class DialogueManager:
    """対談の進行を管理するクラス"""
    def __init__(self, characters: list, topics: list):
        self.characters = characters
        self.topics = topics
        self.shared_history = []

    def _introduce_topic(self, topic: str):
        """トピックを紹介し、最初の発言を促すメッセージを作成する"""
        print("\n" + "="*50)
        print(f"【新たなテーマ】: {topic}")
        print("="*50 + "\n")
        moderator_message = f"司会者: それでは、次のテーマ「{topic}」について議論を始めましょう。"
        self.shared_history.append(HumanMessage(content=moderator_message))
        return f"最初の議題として、「{topic}」について、皆さんのご意見をお聞かせください。"

    # ★修正点2: 過去のチャット履歴を踏まえた対話フローの明確化
    def run_discussion(self, turns_per_character: int = 5):
        """
        全テーマにわたる対談を実行する。
        共有の会話履歴(self.shared_history)を各キャラクターに渡すことで、
        過去の発言の文脈を踏まえた応答を生成させる。
        """
        for topic in self.topics:
            current_input = self._introduce_topic(topic)
            last_speaker = None

            total_turns = len(self.characters) * turns_per_character
            print(f"今回のテーマでは、合計 {total_turns} 回の発言が予定されています。")
            print("-" * 50 + "\n")

            for turn in range(total_turns):
                # 1. 次の発言者を決定
                if last_speaker:
                    possible_speakers = [c for c in self.characters if c.name != last_speaker.name]
                    speaker = random.choice(possible_speakers)
                else:
                    speaker = random.choice(self.characters)

                print(f"--- (ターン{turn + 1}/{total_turns}) {speaker.name}の発言 ---")

                # 2. 発言を生成
                # Character.speakに直前の発言(current_input)と全会話履歴(self.shared_history)を渡す。
                # Character内のチェーンが、これらの情報から文脈を理解して応答を生成する。
                response = speaker.speak(current_input, self.shared_history)
                response = response[:200]

                print(response)
                print("-" * 20 + "\n")

                # 3. 共有履歴を更新
                # 今回の発言を、発言者(name)を明記して履歴に追加。
                # これにより、次の発言者はこの発言もコンテキストとして利用できる。
                self.shared_history.append(AIMessage(content=response, name=speaker.name))

                # 4. 次の入力と発言者を更新
                # 次のキャラクターへの入力を、今回の発言内容そのものに設定し、
                # 自然な会話のキャッチボールを実現する。
                current_input = response
                last_speaker = speaker

# ==============================================================================
# 3. メイン処理
# ==============================================================================
if __name__ == "__main__":
    # スクリプト自身のディレクトリの絶対パスを取得
    # ノートブック環境では__file__が使えないため、os.getcwd()で現在の作業ディレクトリを取得する
    script_dir = os.getcwd()

    character_definitions = [
        {
            "name": "忍者",
            # スクリプトの場所を基準にした絶対パスに変換
            "private_path": os.path.join(script_dir, "忍者/"),
            "shared_path": os.path.join(script_dir, "conversation_logs/"),
            "prompt": "あなたはJリーグのサッカーに詳しい名古屋グランパスが好きな「忍者」です。一人称は「拙者」。「～でござる」「ﾆﾝﾆﾝ」が語尾に付くことが多いです。冷静に、暖かいファン・サポーター目線でどこかユーモラスに名古屋グランパスのことを前向きに語ってください。\n**重要: あなたの発言は常に200文字以内で、要点をまとめて簡潔に話してください。**\n{context}"
        },
        {
            "name": "侍",
            "private_path": os.path.join(script_dir, "侍/"),
            "shared_path": os.path.join(script_dir, "conversation_logs/"),
            "prompt": "あなたはJリーグの名古屋グランパスを好きな「侍」です。一人称は「侍」。「～であろう」「～なかろう」といった武士を思わせる言葉遣いで話します。データと論理に基づき、少し厳しめの視点から、客観的に名古屋グランパスの戦いを分析してください。\n**重要: あなたの発言は常に200文字以内で、要点をまとめて簡潔に話してください。**\n{context}"
        },
        {
            "name": "記者",
            "private_path": os.path.join(script_dir, "記者/"),
            "shared_path": os.path.join(script_dir, "conversation_logs/"),
            "prompt": "あなたはJリーグの名古屋グランパスが好きな記者です。「ですます」体で話します。一人称は省略して話すことが多いです。テーマについて、侍や忍者に対して自分の思う疑問を投げかけて議論を進めることに注力してください。\n**重要: あなたの発言は常に200文字以内で、要点をまとめて簡潔に話してください。**\n{context}"
        }
    ]

    discussion_topics = [
        "名古屋グランパスの2025年度前半（2025年7月まで）の成績をどう評価するのか？",
        "名古屋グランパスの戦術を改善するとしたらなにが必要か？",
        "名古屋グランパスの2025年度後半(2025年8月から12月)になにを期待するか？"
    ]

    # --- 実行 ---
    print("対談シミュレーションを開始します...")
    characters = [
        Character(
            name=c["name"],
            private_docs_path=c["private_path"],
            shared_logs_path=c["shared_path"],
            system_prompt_template=c["prompt"]
        )
        for c in character_definitions
    ]

    manager = DialogueManager(characters, discussion_topics)
    manager.run_discussion(turns_per_character=5)
    print("\n対談シミュレーションを終了します。")

対談シミュレーションを開始します...
キャラクター「忍者」を構築中...
  > 個別知識を 1 件読み込みました。
  > 共有会話ログ '忍者.csv' を読み込み、174 件のドキュメントを取得しました。
  > 合計 175 件のドキュメントで知識ベースを構築します。
キャラクター「忍者」の構築完了。
キャラクター「侍」を構築中...
  > 個別知識を 1 件読み込みました。
  > 共有会話ログ '侍.csv' を読み込み、62 件のドキュメントを取得しました。
  > 合計 63 件のドキュメントで知識ベースを構築します。
キャラクター「侍」の構築完了。
キャラクター「記者」を構築中...
  > 個別知識を 1 件読み込みました。
  > 共有会話ログ '記者.csv' を読み込み、228 件のドキュメントを取得しました。
  > 合計 229 件のドキュメントで知識ベースを構築します。
キャラクター「記者」の構築完了。

【新たなテーマ】: 名古屋グランパスの2025年度前半（2025年7月まで）の成績をどう評価するのか？

今回のテーマでは、合計 15 回の発言が予定されています。
--------------------------------------------------

--- (ターン1/15) 侍の発言 ---
侍: 侍
「成敗！（Say-Bye！）」: 「2025年度前半の名古屋グランパスの成績は、戦術の浸透度や選手のコンディションに大きく左右されるであろう。特に、守備の安定性が欠如していた場合、失点が増え、順位に影響を及ぼすことが予想されるなか、攻撃陣の連携が改善されれば、勝利を重ねる可能性も高まるであろう。冷静にデータを分析し、戦術を見直すことが肝要であるな。」成敗！（Say-Bye！）
--------------------

--- (ターン2/15) 忍者の発言 ---
忍者: 忍者
「成敗！（Say-Bye！）」: 「2025年度前半の名古屋グランパス、評価は難しいでござるが、攻撃陣の連携が鍵を握るでござるな。守備が安定せず失点が多ければ、順位が下がることも。また、新戦力の活躍が期待される中、選手のコンディションも重要でござる。冷静にデータを分析し、戦術を見直すことが、さらなる飛躍へと繋がるはずでござるよ。ﾆﾝ

KeyboardInterrupt: 

In [ ]:
!git clone https://github.com/YukinobuYoshihara/yawarakame.git

In [ ]:
import os
# 作成されたリポジトリのディレクトリに移動する
os.chdir('yawarakame')